In [ ]:
"""All TODO"""
# TODO: Add ground-truth data stations as different shape / colour nodes
# TODO: Plot all timeseries data in full, knock out two unacceptable ones and then plot omitting suspect and unchecked data
# TODO: interpolate missing data (with quick missingness checks - check DEVUL assignment feedback on this)
# TODO: GNN iteration 1 inc. shallow aquifer then test metrics to decide on inclusion (RMSE, MSE etc). See Notion: Project Data > Preprocessing Steps > 0b. > Full Notes (Page)

Imports

In [ ]:
import os
import ast
import folium
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from pyproj import Transformer
from shapely.geometry import box
from datetime import datetime, timedelta

In [ ]:
# print("pyproj:", pyproj.__version__)
# print("shapely:", shapely.__version__)

Mesh building function definition

Create mesh using input shape file

Folium interactive map (open from html file for full view). Catchment boundary currently interactive.

Basic matplotlib map

Import Station Data using DEFRA API

**API Documentation notes:**

1. The API calls that return readings data have a soft limit of 100,000 rows per-call which can be overridden by setting a _limit parameter. There is a hard limit of 2,000,000 rows, which cannot be overridden.
2. The primary identifier for most stations uses a GUID style identifier called an SUID. These are used in the URL for the station and given as the value of the notation property in the station metadata.  
    a. Wiski identifier (wiskiID) is also available for my subset of stations and data type  
3. All monitoring stations can be filtered by name, location and other parameters. See https://environment.data.gov.uk/hydrology/doc/reference#stations-summary for full metadata details

Convert raw csv files into catchment dict

In [ ]:
def load_timeseries_to_dict(stations_df, col_order, data_dir="data/gwl/"):
    """
    Loads and cleans groundwater level timeseries data from CSV files.
    
    - Removes 'qcode' column if present.
    - Ensures all columns in `col_order` are present (filling missing with NA).
    - Reorders columns to match `col_order`.
    - Returns a dictionary of cleaned DataFrames keyed by station name.
    """ 
    # Save pandas dataframes to a dictionary by station name
    time_series_data = {}

    for index, row in stations_df.iterrows():
        uri = row['measure_uri']
        measure_id = uri.split("/")[-1]
        name = row['station_name'].title().strip().replace(" ", "_")
        
        # Read CSV into placeholder df to manipulate
        temp_df = pd.read_csv(f"{data_dir}{measure_id}_readings.csv", index_col=0, low_memory=False)
        
        # Drop 'qcode' column if present
        if 'qcode' in temp_df.columns:
            temp_df = temp_df.drop(columns=['qcode'])
        
        # Reorder columns (fill missing with NA)
        for col in col_order:
            if col not in temp_df.columns:
                print(f'Warning: {name} did not contain {col}')
                temp_df[col] = pd.NA
        temp_df = temp_df[col_order]
        
        # Save to dictionary
        time_series_data[name] = temp_df
        
    return time_series_data

# Load timeseries CSVs from API into reference dict
col_order = ['station_name', 'date', 'dateTime', 'value', 'quality', 'measure']
time_series_data = load_timeseries_to_dict(stations_df, col_order)

Remove outliers and resample gwl to daily resolution

In [ ]:
def remove_outliers(df, z_thresh=4):
    # Ensure numeric
    df = df.copy()
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    
    # Z-score outlier removal
    z = (df['value'] - df['value'].mean()) / df['value'].std()
    df.loc[z.abs() > z_thresh, 'value'] = pd.NA
    return df

def resample_daily_average(df):
    df = df.copy()
    df['dateTime'] = pd.to_datetime(df['dateTime'], errors='coerce')
    df = df.dropna(subset=['dateTime'])
    df = df.sort_values('dateTime')

    # Set index for resampling
    df = df.set_index('dateTime')
    
    # Define aggregation functions
    agg_funcs = {
        'station_name': 'first',
        'date': 'first',
        'value': 'mean',
        'quality': lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA,
        'measure': 'first'
    }
    
    # Resample and aggregate
    daily_df = df.resample('1D').agg(agg_funcs).reset_index()
    
    return daily_df

# Clean data and resample to days
daily_time_series = {}
for station, df in time_series_data.items():
    clean_df = remove_outliers(df)
    daily_avg_df = resample_daily_average(clean_df)
    daily_time_series[station] = daily_avg_df

Plot initial cleaned data as time series line graphs to begin to understand the data

In [ ]:
def plot_timeseries(time_series_raw, station_name):
    """Resusable matplotlib time series plot"""
    fig, ax = plt.subplots(figsize=(15, 4))
    time_series_raw['dateTime'] = pd.to_datetime(time_series_raw['dateTime'], errors='coerce')

    # Define fixed colours for each quality level
    quality_colors = {
        'Good': '#70955F',
        'Estimated': '#549EB1',
        'Suspect': '#DF6607',
        'Unchecked': '#e89c1d',
        'Missing': '#9c9acd'
    }
    
    # Plot using qualities score as legend
    for quality, color in quality_colors.items():
        temp = time_series_raw.copy()
        temp['value'] = temp['value'].where(temp['quality'] == quality, pd.NA)
        ax.plot(temp['dateTime'], temp['value'], label=quality, color=color, alpha=0.8)

    # Apply auto locators and formatters to clean up ticks
    locator = mdates.AutoDateLocator(minticks=10)
    formatter = mdates.ConciseDateFormatter(locator)
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(formatter)
    fig.autofmt_xdate()
    
    ax.set_title(f'{station_name} Groundwater Level 2014-2025')
    ax.set_xlabel('Date')
    ax.set_ylabel('Groundwater Level (mAOD)')
    ax.grid()
    ax.legend(title="Quality", loc="center left", bbox_to_anchor=(1.01, 0.5))
    
    plt.tight_layout()
    plt.savefig(f"figures/eden_catchment/raw_timeseries_plots/{station_name}_raw_plot_.png", dpi=300)
    plt.show()

# Plot data at a daily resolution
for station in daily_time_series:
    print(f'Columns for {station}:\n    {daily_time_series[station].columns}\n    Total Entries: {len(daily_time_series[station])}\n')
    plot_timeseries(daily_time_series[station], station)

Create geodataframes for stations:

In [ ]:
# from shapely.geometry import Point

# # Convert to GeoDataFrame using WGS84 (lat/lon)
# stations_gdf = gpd.GeoDataFrame(
#     stations_df,
#     geometry=gpd.points_from_xy(stations_df['lon'], stations_df['lat']),
#     crs="EPSG:4326"
# )

# # Reproject to match mesh CRS (British National Grid)
# stations_gdf = stations_gdf.to_crs("EPSG:27700")


Snap stations to nearest mesh node

In [ ]:
# from shapely.ops import nearest_points
# import shapely.geometry

# # Rebuild mesh index
# mesh_sindex = mesh_nodes_gdf.sindex

# def find_nearest_node(station_point):
#     # This is the key fix: pass geometry directly, not as a list
#     nearest_idx = list(mesh_sindex.nearest(station_point, return_all=False))[0]

#     nearest_row = mesh_nodes_gdf.iloc[nearest_idx]
#     nearest_geom = nearest_row.geometry

#     # Ensure it's a proper Shapely Point
#     if hasattr(nearest_geom, '__geo_interface__') and not isinstance(nearest_geom, shapely.geometry.base.BaseGeometry):
#         nearest_geom = shapely.geometry.shape(nearest_geom)

#     return pd.Series({
#         'nearest_node_id': int(nearest_row['node_id']),
#         'nearest_geometry': nearest_geom
#     })


# # Apply snapping
# stations_gdf[['nearest_node_id', 'nearest_geometry']] = stations_gdf.geometry.apply(find_nearest_node)
# stations_gdf = stations_gdf.set_geometry('nearest_geometry').to_crs("EPSG:4326")
# stations_gdf['lat'] = stations_gdf.geometry.y
# stations_gdf['lon'] = stations_gdf.geometry.x


In [ ]:
# print(stations_gdf[['station_id', 'nearest_node_id', 'lat', 'lon']])

In [ ]:
# import folium

# # Recreate map centered on catchment
# map_center = [mesh_nodes_gdf['lat'].mean(), mesh_nodes_gdf['lon'].mean()]
# map = folium.Map(location=map_center, zoom_start=10, tiles="CartoDB positron")

# # Add snapped station locations
# for _, row in stations_gdf.iterrows():
#     folium.Marker(
#         location=[row['lat'], row['lon']],
#         popup=f"{row['station_id']} → Node {row['nearest_node_id']}"
#     ).add_to(map)

# # Optionally add mesh nodes
# for _, row in mesh_nodes_gdf.iterrows():
#     folium.CircleMarker(
#         location=[row['lat'], row['lon']],
#         radius=1,
#         color="#354c7c",
#         fill=True,
#         fill_opacity=0.6
#     ).add_to(map)

# # Save
# map.save("figures/station_to_mesh_snapping.html")
# map
